In [1]:
from LorisBallsBasedModel import *
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
cols = ['price', 'maint', 'doors', 'persons', 'lug_capacity', 'safety','output']
cars = pd.read_csv(r'data\car_evaluation_dataset\car_evaluation.csv', names=cols, header=None)
cars.head(3)

,price,maint,doors,persons,lug_capacity,safety,output
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc


In [3]:
labels = pd.get_dummies(cars.output, prefix='condition')

X = cars[['price', 'maint', 'doors', 'persons', 'lug_capacity', 'safety']]

y = labels.values

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.20,
                                                    random_state=42)

# SingleLayerPerceptron base model

In [5]:
X_train_embedding_layers = {}
for input_name in X_train.columns:
    X_train_embedding_layers[input_name] = StringEmbedding(X_train[input_name].unique(),
                                                           len(X_train[input_name].unique()))

processing_layer = InputsProcessing(X_train_embedding_layers)
    
X_train_dict = X_train.to_dict(orient='list')
X_train_dict = {k: np.expand_dims(v, axis=-1) for k, v in X_train_dict.items()}
X_test_dict = X_test.to_dict(orient='list')
X_test_dict = {k: np.expand_dims(v, axis=-1) for k, v in X_test_dict.items()}

train_tensor = tf.data.Dataset.from_tensor_slices((X_train_dict,
                                                   y_train))
train_tensor = train_tensor.cache().shuffle(300).batch(8).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [6]:
SLP = SingleLayerPerceptron(tf.keras.layers.Dense(4,
                                                  activation='softmax'),
                            processing_layer=processing_layer)
SLP.compile(loss='categorical_crossentropy',
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
            metrics=['acc'])

In [7]:
history = SLP.fit(train_tensor, epochs=10, verbose=1)

Epoch 1/10
173/173 [==============================] - 1s 2ms/step - loss: 1.1519 - acc: 0.6686
Epoch 2/10
173/173 [==============================] - 0s 2ms/step - loss: 0.7732 - acc: 0.7055
Epoch 3/10
173/173 [==============================] - 0s 2ms/step - loss: 0.6185 - acc: 0.7062
Epoch 4/10
173/173 [==============================] - 0s 2ms/step - loss: 0.5078 - acc: 0.7699
Epoch 5/10
173/173 [==============================] - 0s 2ms/step - loss: 0.4318 - acc: 0.8133
Epoch 6/10
173/173 [==============================] - 0s 2ms/step - loss: 0.3817 - acc: 0.8531
Epoch 7/10
173/173 [==============================] - 0s 2ms/step - loss: 0.3465 - acc: 0.8741
Epoch 8/10
173/173 [==============================] - 0s 2ms/step - loss: 0.3195 - acc: 0.8813
Epoch 9/10
173/173 [==============================] - 0s 1ms/step - loss: 0.2986 - acc: 0.8842
Epoch 10/10
173/173 [==============================] - 0s 2ms/step - loss: 0.2816 - acc: 0.8878


In [8]:
test_tensor = tf.data.Dataset.from_tensor_slices((X_test_dict,
                                                  y_test)).batch(8)
SLP.evaluate(test_tensor)

44/44 [==============================] - 0s 2ms/step - loss: 0.2920 - acc: 0.8786


[0.29203733801841736, 0.8786126971244812]